In [32]:
from pyairtable import Api
from dotenv import load_dotenv
import os
load_dotenv()

api_key = os.getenv('AIRTABLE_API_KEY')
base_id = os.getenv('AIRTABLE_BASE')

api = Api(api_key)

matches_table = api.table(base_id, 'matches')
people_table = api.table(base_id, 'people')

In [33]:
# read from mail_template.txt
mail_template = ''
with open('mail_template.txt', 'r') as file:
    mail_template = file.read()

# replace {{name}} with the name of the person

In [29]:
import base64
from email.mime.text import MIMEText
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from requests import HTTPError

SCOPES = [
        "https://www.googleapis.com/auth/gmail.send"
    ]
flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
creds = flow.run_local_server(port=0)

service = build('gmail', 'v1', credentials=creds)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725730991270-om13lfcjhpoc2ptl1g4ajhoklgt0j6qf.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A49934%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=Lti35MstsPefRLXUpTGAWhY5FSWgPY&access_type=offline


In [30]:
import base64
import markdown
from tqdm import tqdm
import time

In [34]:
subject_template = mail_template.split('\n')[0].split('Subject: ')[1]
mail_template = "\n".join(mail_template.split('\n')[2:])
mail_template = markdown.markdown(mail_template)

In [ ]:
bool = False
for record in tqdm(people_table.all()):

    email = mail_template.replace('{{name}}', record['fields']['name'].split(' ')[0])

    print(F'sending email to {record["fields"]["email"]}')
    message = MIMEText(email.replace('\n', '<br>'), 'html')
    message['to'] = record['fields']['email']
    message['subject'] = subject_template
    create_message = {'raw': base64.urlsafe_b64encode(message.as_bytes()).decode()}

    try:
        message = (service.users().messages().send(userId="me", body=create_message).execute())
        print(F'sent message to {message} Message Id: {message["id"]}')
    except HTTPError as error:
        print(F'An error occurred: {error}')
        message = None

    time.sleep(1) # avoid rate limit